# Collecting the parcel data from King County
_Note, this is made for condominiums (many units in one parcel), but could easily be modified for other use cases._

[parcelviewer2](https://gismaps.kingcounty.gov/parcelviewer2/) is here. You choose the parcel that you want on this map. 

Then you enter your parcel into the field to get the parcel that you want.
The data for your parcel is here: 
```
https://blue.kingcounty.com/Assessor/eRealProperty/Dashboard.aspx?ParcelNbr={yourparcel#}
```

That contains a table of all of the units in that parcel. Copy the parcel numbers in the table under `Units in this condominium complex` into a text file: `data/parcel_numbers.txt`. 

**Note**, I had to clean that using VSC's search and replace. I used the pattern `[^0-9]`.





In [1]:
import os,yaml
import requests, urllib
from bs4 import BeautifulSoup as bs
import pandas as pd



In [2]:
params = yaml.safe_load(open('config.yaml'))

with open(os.path.join('..','data',params['parcel_numbers'])) as f:
    lines = f.read().splitlines()

print(f'found: {len(lines)} parcels in file')

found: 84 parcels in file


In [3]:
URL_ParcelNumberQuery = r"https://blue.kingcounty.com/Assessor/eRealProperty/Dashboard.aspx?ParcelNbr={}"

The process below is most likely to change with updates to the website. See the comment `likely need to update`.  I could automate this better but it's not something that I need to do except _at most_ once a year. However you can just update by cicling through the index of `tables` to find the table that you want.

`pd.read_html(str(tables), header=0)[X]` will give you a glimpse of your table where `X` is the number of the table you want to see. 

In [9]:
from re import template


condo_tables = []
value_tables = []
counter = 0

for parcel in lines[:20]:
    if counter % 5 == 0:
        print(counter,end=' ')
    page = urllib.request.urlopen(URL_ParcelNumberQuery.format(parcel))
    myhtml = page.read().decode('utf-8')
    soup = bs(myhtml)
    tables = soup.find_all('table')
    # likely need to update based on where the table is on the webpage
    # in 10/2022 the tables I wanted are on 6, and 8 below.
    tempTable = pd.read_html(str(tables), header=0)[6]
    lotKey = pd.DataFrame(pd.read_html(str(tables), header=0)[12]).set_index('Parcel Number').to_dict()
    par_cel=list(lotKey.keys())[0]
    lotdict=lotKey[parcel]
    lotdict['Name']
    lotdict['Site Address']
    lotdict['Legal']
    condo_tables.append(tempTable)
    counter += 1


0 5 10 15 

In [10]:
pd.concat(value_tables)

,Valued Year,Tax Year,Appraised Land Value ($),Appraised Imps Value ($),Appraised Total ($),Appraised Imps Increase ($),Taxable Land Value ($),Taxable Imps Value ($),Taxable Total ($)
0,2022,2023,145300,226700,372000,0,145300,226700,372000
1,2021,2022,145300,217700,363000,0,145300,217700,363000
2,2020,2021,145300,165700,311000,0,145300,165700,311000
3,2019,2020,133200,147800,281000,0,133200,147800,281000
4,2018,2019,133200,149800,283000,0,133200,149800,283000
...,...,...,...,...,...,...,...,...,...
29,1990,1991,0,0,0,0,11800,38700,50500
30,1988,1989,0,0,0,0,10100,33100,43200
31,1986,1987,0,0,0,0,9400,44200,53600
32,1984,1985,0,0,0,0,9400,44200,53600


In [48]:
lotKey = pd.DataFrame(pd.read_html(str(tables), header=0)[12]).set_index('Parcel Number').to_dict()
par_cel=list(lotKey.keys())[0]
lotdict=lotKey[parcel]
lotdict['Name']
lotdict['Site Address']
lotdict['Legal']


{'Name': 'CLARK VICTORIA A',
 'Site Address': '3711 26TH PL W 304 98199',
 'Legal': 'WINDY HILLS CONDOMINIUM APT 304 & P-31 PCT OF VAL 1.24'}

In [24]:
parcel

'9478110200'